In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm

In [3]:
structures = pd.read_csv("datasets/dichalcogenides_innopolis_202105/defects.csv", index_col=0)

In [4]:
defects_descriptors = pd.read_csv("datasets/dichalcogenides_innopolis_202105/descriptors.csv", index_col=0)

In [5]:
import pymatgen.io.cif
STRUCTURES_FOLDER = "datasets/dichalcogenides_innopolis_202105/initial/"
initial_structures = dict()
for structure_file in tqdm(os.listdir(STRUCTURES_FOLDER)):
  this_file = pymatgen.io.cif.CifParser(os.path.join(STRUCTURES_FOLDER, structure_file))
  initial_structures[os.path.splitext(structure_file)[0]] = this_file.get_structures()[0]

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [6]:
structures["initial_structure"] = structures.apply(lambda row: initial_structures[row._id], axis=1)

In [7]:
import matplotlib
import pylab

from math import floor, ceil
from random import shuffle, seed
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

cm = pylab.get_cmap('Set1')

import tensorflow as tf
import numpy as np

In [8]:
import wandb
from wandb.keras import WandbCallback

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [11]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [12]:
config = wandb.config
config.target = "energy"

In [13]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc)#.from_file('8.9e-4')

In [14]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=False)
model.target_scaler = scaler

In [ ]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=100, verbose=1, patience=1000)

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 177s 8s/step - loss: 1.1438


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 19.48412, saving model to callback/val_mae_00001_19.484121.hdf5


Epoch 2/100
21/21 [==============================] - 95s 4s/step - loss: 0.6201


...

Epoch 10/100
21/21 [==============================] - 138s 6s/step - loss: 0.7962
Epoch 11/100
21/21 [==============================] - 204s 9s/step - loss: 0.6437
Epoch 12/100
21/21 [==============================] - 162s 8s/step - loss: 0.6947
Epoch 13/100
21/21 [==============================] - 133s 6s/step - loss: 1.3745
Epoch 14/100
21/21 [==============================] - 200s 10s/step - loss: 0.6825
Epoch 15/100
21/21 [==============================] - 154s 7s/step - loss: 1.0191
Epoch 16/100
21/21 [==============================] - 160s 7s/step - loss: 0.4308
Epoch 17/100
21/21 [==============================] - 125s 6s/step - loss: 0.7270
Epoch 18/100
21/21 [==============================] - 136s 6s/step - loss: 0.6391
Epoch 19/100
21/21 [==============================] - 173s 8s/step - loss: 0.6428
Epoch 20/100
21/21 [==============================] - 163s 8s/step - loss: 0.7865
Epoch 21/100
21/21 [==============================] - 176s 8s/step - loss: 0.5696


INFO:megnet.callbacks:
Epoch 00021: val_mae improved from 3.96739 to 3.73976, saving model to callback/val_mae_00021_3.739757.hdf5


Epoch 22/100
21/21 [==============================] - 172s 8s/step - loss: 0.6591


INFO:megnet.callbacks:
Epoch 00022: val_mae improved from 3.73976 to 3.50279, saving model to callback/val_mae_00022_3.502786.hdf5


Epoch 23/100
21/21 [==============================] - 114s 5s/step - loss: 0.7576
Epoch 24/100
21/21 [==============================] - 108s 5s/step - loss: 0.5800
Epoch 25/100
21/21 [==============================] - 117s 6s/step - loss: 0.4861
Epoch 26/100
21/21 [==============================] - 138s 7s/step - loss: 0.5266
Epoch 27/100
21/21 [==============================] - 170s 8s/step - loss: 0.5398
Epoch 28/100
21/21 [==============================] - 222s 11s/step - loss: 0.6556
Epoch 29/100
21/21 [==============================] - 211s 10s/step - loss: 0.9967
Epoch 30/100
21/21 [==============================] - 202s 10s/step - loss: 0.8546
Epoch 31/100
21/21 [==============================] - 183s 9s/step - loss: 0.7334
Epoch 32/100
21/21 [==============================] - 171s 8s/step - loss: 1.0070
Epoch 33/100
21/21 [==============================] - 121s 6s/step - loss: 0.6370
Epoch 34/100
21/21 [==============================] - 150s 7s/step - loss: 0.6155
Epoch 35/100


In [ ]:
# TODO(kazeevn) speedup
test.loc[:, "predicted"] = model.predict_structures(test["initial_structure"].values)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(test[config.target], test["predicted"])
ax.set_xlabel("True energy, eV")
ax.set_ylabel("Predicted energy, eV");